In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep
import pandas as pd

In [2]:
version = "1.0.0"

# Get the article list

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [4]:
def gedjournalarticles(start_url, journaltitle, filename ):
    # open old file
    olddf = pd.DataFrame({'url': [], 'journal': [] })
    try:
        olddf = pd.read_csv(filename, sep="|")
        print(olddf.tail())
    except:
        pass
    
    driver.get(start_url)
    print(driver.current_url)
    
    # itterate over the journal searching to get the Article list
    articles = []
    def pagefind():
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # found the Article
        for h5 in soup.find_all("h5"):
            # get link 
            for a in h5.find_all("a"):        
                articles.append("https://www.cambridge.org"+a['href'])
    
        # find next page
        for ulp in driver.find_elements_by_class_name('pagination'):    
            for lis in ulp.find_elements_by_tag_name('a'):
                if re.search('Next', lis.text):
                    # go to the next page
                    lis.click()
                    sleep(2)
                    return True
        return False

    nextpage = True
    while  nextpage:
        nextpage = pagefind()                
        print(driver.current_url, end='\r')
        
    # filer out old articles
    oldarticles = olddf['url']
    articles = list( set(articles).difference(set(oldarticles) ))
    print(len(articles), "new article found!")
    
    # make df
    df = pd.DataFrame({'url': list(set(articles)), 'journal': journaltitle })
    
    return df, olddf

# Process new articles

In [5]:
def getarticledetails(df, olddf, filename):
    
    articles = df['url'].values

    titles = [ None for _ in range(len(df))]
    abstracts = [ None for _ in range(len(df))]
    writers = [ None for _ in range(len(df))]
    dates = [ None for _ in range(len(df))]
    dois = [  None for _ in range(len(df)) ]
    keywords = [  None for _ in range(len(df)) ]

    for idx in range(len(articles)):
        # print percentiage of the process
        print( str(np.round(100*idx/len(df),2))+"%" , end='\r')
    
        url = df.iloc[idx]['url']
        request = urllib.request.Request(url)
        request.add_header('Accept-Encoding', 'utf-8')
        try:
            response = urlopen(request)    
            page_content = response.read().decode('utf-8')
        except:
            continue
    
        page_soup = BeautifulSoup(page_content, features="lxml", from_encoding='utf-8')
        
        # abstract
        abstract = ""
        for abstdiv in page_soup.findAll("div", {'class':['abstract']}):
            for ap in abstdiv.find_all("p"):
                abstract = abstract + (" ".join(ap.get_text().splitlines()).replace("\xa0", " ") )
        if len(abstract) > 0:
            abstracts[idx] = abstract
        else:
            # if no abstract we ignore the page
            continue

        # title
        for t in page_soup.find_all('title'):        
            titles[idx] = t.get_text().split("|")[0]                
            
        # writer and publish date
        writter = []
        for m in page_soup.find_all("meta"):
            if m.has_attr("name"):
                if m['name'] == "citation_author":
                    writter.append( m['content'] )
                # add the author institute to the author
                elif m['name'] == "citation_author_institution":
                    writter[-1] = writter[-1] + "--" + m['content']
                # add writer orcid 
                elif m['name'] == "citation_author_orcid":
                    writter[-1] = writter[-1] + "---" + m['content']
                elif m['name'] == "citation_online_date":
                    # change to ISO dateformat
                    dates[idx] = datetime.datetime.strptime(m['content'], "%Y/%m/%d").strftime('%Y-%m-%d')
                elif m['name'] == "citation_doi":
                    dois[idx] = m['content']
        if len(writter) > 0:
            writers[idx] =  "#".join(writter)
        
        # keywords
        key = []
        for keya in page_soup.findAll("a", {'class':['keywords']}):
            key.append(keya.text)
        if len(key) > 0:
            keywords[idx] =  "#".join(key)
    
    # extend the df
    df['title'] = titles
    df['doi'] = dois
    df['abstract'] = abstracts
    df['writer'] = writers
    df['publishdate'] = dates
    df['keyword'] = keywords
    
    # merge owith old df
    df = pd.concat([df, olddf])
    
    # save data
    df.to_csv(filename, sep="|", index=False)
    
    # test
    test = pd.read_csv(filename, sep="|")
    print(test.tail())

# Main

In [6]:
journals = ['Annals_of_Glaciology', 'Journal_of_Glaciology']
start_urls = [ 
    "https://www.cambridge.org/core/journals/annals-of-glaciology/listing?q=&searchWithinIds=DD7BD3FCBF82BF09609E5A0E1888ED7B&aggs[productTypes][filters]=JOURNAL_ARTICLE",
    "https://www.cambridge.org/core/journals/journal-of-glaciology/listing?q=&searchWithinIds=FE8284B2577ADBCD299821FDE4E752F1&aggs[productTypes][filters]=JOURNAL_ARTICLE"
]

In [7]:
for jidx in range(len(journals)):
    filename = filename = 'data/journal_'+ journals[jidx] +'_'+version+'.csv'
    
    # search for articles
    df, olddf = gedjournalarticles(start_urls[jidx], journals[jidx], filename )
    
    # get the articles details
    getarticledetails(df, olddf, filename)

                                                    url               journal  \
3843  https://www.cambridge.org/core/journals/annals...  Annals_of_Glaciology   
3844  https://www.cambridge.org/core/journals/annals...  Annals_of_Glaciology   
3845  https://www.cambridge.org/core/journals/annals...  Annals_of_Glaciology   
3846  https://www.cambridge.org/core/journals/annals...  Annals_of_Glaciology   
3847  https://www.cambridge.org/core/journals/annals...  Annals_of_Glaciology   

                                                  title  \
3843  An initial numerical investigation of the exte...   
3844  A synthesis of remote sensing data on Wilkins ...   
3845  AVHRR-based Polar Pathfinder products for mode...   
3846  The potential of using Landsat 7 ETM+ for the ...   
3847  Variability in accumulation rates from GPR pro...   

                             doi  \
3843   10.3189/1991AoG15-1-31-36   
3844   10.3189/S0260305500012866   
3845   10.3189/S0260305500014336   
3846  10.3189/

/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


                                                    url  \
5532  https://www.cambridge.org/core/journals/journa...   
5533  https://www.cambridge.org/core/journals/journa...   
5534  https://www.cambridge.org/core/journals/journa...   
5535  https://www.cambridge.org/core/journals/journa...   
5536  https://www.cambridge.org/core/journals/journa...   

                    journal  \
5532  Journal_of_Glaciology   
5533  Journal_of_Glaciology   
5534  Journal_of_Glaciology   
5535  Journal_of_Glaciology   
5536  Journal_of_Glaciology   

                                                  title  \
5532                                    Zusammenfassung   
5533                                    Zusammenfassung   
5534  A combined surface-and volume-scattering model...   
5535                                    Zusammenfassung   
5536  Spatial and temporal variations in albedo on S...   

                             doi  \
5532   10.3189/S0022143000018931   
5533   10.3189/S002214300000683